In [1]:
import numpy as np
from collections import deque, namedtuple
from dataclasses import dataclass

In [2]:
@dataclass
class Blizzard:
    y: int
    x: int
    direction: int
        
    def get_loc(self, time, grid_size):
        h, w = grid_size
        if self.direction == 0:
            x = self.x - 1
            x = (x + time) % (w - 2) + 1
            return (self.y, x)
        elif self.direction == 1:
            y = self.y - 1
            y = (y + time) % (h - 2) + 1
            return (y, self.x)
        elif self.direction == 2:
            x = self.x - 1
            x = (x - time) % (w - 2) + 1
            return (self.y, x)
        elif self.direction == 3:
            y = self.y - 1
            y = (y - time) % (h - 2) + 1
            return (y, self.x)
        
State = namedtuple('State', 'time,player_y,player_x')

In [3]:
with open('input.txt') as f:
    data = f.read().rstrip()
    grid = np.array([list(line) for line in data.splitlines()])

In [4]:
h, w = grid.shape
empty_grid = grid.copy()
empty_grid[1:h-1, 1:w-1] = '.'

In [5]:
blizzards = []

for (y, x) in np.argwhere(grid == '>'):
    blizzards.append(Blizzard(y, x, 0))

for (y, x) in np.argwhere(grid == 'v'):
    blizzards.append(Blizzard(y, x, 1))
    
for (y, x) in np.argwhere(grid == '<'):
    blizzards.append(Blizzard(y, x, 2))
    
for (y, x) in np.argwhere(grid == '^'):
    blizzards.append(Blizzard(y, x, 3))

In [6]:
def get_grid(time, blizzards, empty_grid):
    new_grid = empty_grid.copy()
    for b in blizzards:
        y, x = b.get_loc(time, new_grid.shape)
        if new_grid[y, x] in ('>', 'v', '<', '^'):
            new_grid[y, x] = '2'
        elif new_grid[y, x].isdigit():
            new_grid[y, x] = str(int(new_grid[y, x]) + 1)
        else:
            new_grid[y, x] = ('>', 'v', '<', '^')[b.direction]
    return new_grid

In [7]:
def get_time(start, target, empty_grid, blizzards, start_time=0):
    queue = deque([State(start_time, start[0], start[1])])
    visited = set()

    time_dict = {}

    while len(queue) > 0:
        time,player_y,player_x = queue.popleft()

        if (time,player_y,player_x) in visited:
            continue
        else:
            visited.add((time,player_y,player_x))

        if (player_y, player_x) == target:
            return time

        if time + 1 in time_dict:
            next_grid = time_dict[time + 1]
        else:
            next_grid = get_grid(time+1, blizzards, empty_grid)
            time_dict[time + 1] = next_grid

        if player_y - 1 > -1 and next_grid[player_y-1, player_x] == '.':
            queue.append(State(time+1, player_y-1, player_x))

        if player_y + 1 < h and next_grid[player_y+1, player_x] == '.':
            queue.append(State(time+1, player_y+1, player_x))

        if player_x - 1 > -1 and next_grid[player_y, player_x-1] == '.':
            queue.append(State(time+1, player_y, player_x-1))

        if player_x + 1 < w and next_grid[player_y, player_x+1] == '.':
            queue.append(State(time+1, player_y, player_x+1))

        if next_grid[player_y, player_x] == '.':
            queue.append(State(time+1, player_y, player_x))

In [8]:
start = (0, np.argwhere(grid[0] == '.').flatten()[0])
target = (h-1, np.argwhere(grid[h-1] == '.').flatten()[0])

## part 1

In [9]:
get_time(start, target, empty_grid, blizzards)

373

## part 2

In [10]:
time1 = get_time(start, target, empty_grid, blizzards)
time2 = get_time(target, start, empty_grid, blizzards, start_time=time1)
time3 = get_time(start, target, empty_grid, blizzards, start_time=time2)
time3

997